In [1]:
import pandas as pd

In [2]:
#pd.set_option("display.max_colwidth", 0)

In [3]:
crowd = pd.read_csv('../data/crowd/10JanRelationsResults.csv')
crowd.head()

,HITId,HITTypeId,Title,Description,Keywords,Reward,CreationTime,MaxAssignments,RequesterAnnotation,AssignmentDurationInSeconds,...,Input.Drug_Brand_Name,Input.atc_label,Input.ATC_LEVEL_0,Input.DieaseName,Input.WordCount,Input.Set_ID,Input.Section,Answer.category.label,Approve,Reject
0,301KG0KX9CLEJJQDZTVL1TQJDT3H2D,35NE15Q62B2VZKMWZRZMWJ35EHVT7Y,Identify the relation between a drug and disease,We need your help in categorizing the type of ...,"terms, classify, group",$0.05,Tue Dec 10 12:11:58 PST 2019,5,BatchId:3865212;OriginalHitTemplateId:920937336;,60,...,Ciclopirox,D01AE14,D,epilepsy,228,2c67dd09-ab1f-4ccf-97b5-421356f64f22,Indication,Indication: treatment,NaN,NaN
1,301KG0KX9CLEJJQDZTVL1TQJDT3H2D,35NE15Q62B2VZKMWZRZMWJ35EHVT7Y,Identify the relation between a drug and disease,We need your help in categorizing the type of ...,"terms, classify, group",$0.05,Tue Dec 10 12:11:58 PST 2019,5,BatchId:3865212;OriginalHitTemplateId:920937336;,60,...,Ciclopirox,D01AE14,D,epilepsy,228,2c67dd09-ab1f-4ccf-97b5-421356f64f22,Indication,Contraindication,NaN,NaN
2,301KG0KX9CLEJJQDZTVL1TQJDT3H2D,35NE15Q62B2VZKMWZRZMWJ35EHVT7Y,Identify the relation between a drug and disease,We need your help in categorizing the type of ...,"terms, classify, group",$0.05,Tue Dec 10 12:11:58 PST 2019,5,BatchId:3865212;OriginalHitTemplateId:920937336;,60,...,Ciclopirox,D01AE14,D,epilepsy,228,2c67dd09-ab1f-4ccf-97b5-421356f64f22,Indication,Effect,NaN,NaN
3,301KG0KX9CLEJJQDZTVL1TQJDT3H2D,35NE15Q62B2VZKMWZRZMWJ35EHVT7Y,Identify the relation between a drug and disease,We need your help in categorizing the type of ...,"terms, classify, group",$0.05,Tue Dec 10 12:11:58 PST 2019,5,BatchId:3865212;OriginalHitTemplateId:920937336;,60,...,Ciclopirox,D01AE14,D,epilepsy,228,2c67dd09-ab1f-4ccf-97b5-421356f64f22,Indication,Contraindication,NaN,NaN
4,301KG0KX9CLEJJQDZTVL1TQJDT3H2D,35NE15Q62B2VZKMWZRZMWJ35EHVT7Y,Identify the relation between a drug and disease,We need your help in categorizing the type of ...,"terms, classify, group",$0.05,Tue Dec 10 12:11:58 PST 2019,5,BatchId:3865212;OriginalHitTemplateId:920937336;,60,...,Ciclopirox,D01AE14,D,epilepsy,228,2c67dd09-ab1f-4ccf-97b5-421356f64f22,Indication,I dont know,NaN,NaN


In [4]:
crowd.columns

Index(['HITId', 'HITTypeId', 'Title', 'Description', 'Keywords', 'Reward',
       'CreationTime', 'MaxAssignments', 'RequesterAnnotation',
       'AssignmentDurationInSeconds', 'AutoApprovalDelayInSeconds',
       'Expiration', 'NumberOfSimilarHITs', 'LifetimeInSeconds',
       'AssignmentId', 'WorkerId', 'AssignmentStatus', 'AcceptTime',
       'SubmitTime', 'AutoApprovalTime', 'ApprovalTime', 'RejectionTime',
       'RequesterFeedback', 'WorkTimeInSeconds', 'LifetimeApprovalRate',
       'Last30DaysApprovalRate', 'Last7DaysApprovalRate', 'Input.No',
       'Input.Label_ID', 'Input.From', 'Input.To', 'Input.Type',
       'Input.Annotation', 'Input.Context', 'Input.DO_ID', 'Input.DB_ID',
       'Input.DrugName', 'Input.Drug_Brand_Name', 'Input.atc_label',
       'Input.ATC_LEVEL_0', 'Input.DieaseName', 'Input.WordCount',
       'Input.Set_ID', 'Input.Section', 'Answer.category.label', 'Approve',
       'Reject'],
      dtype='object')

In [5]:
print (len(crowd))
#crowd = crowd.dropna(subset=['label_id'])
print (len(crowd))

3865
3865


In [6]:
columns= ['Input.Label_ID', 'Input.From', 'Input.To', 'Input.Type',
       'Input.Annotation', 'Input.Context', 'Input.DO_ID', 'Input.DB_ID',
       'Input.DrugName', 'Input.Drug_Brand_Name', 'Input.atc_label',
       'Input.ATC_LEVEL_0', 'Input.DieaseName', 'Input.WordCount',
       'Input.Set_ID', 'Input.Section']
df = crowd.groupby(columns)['Answer.category.label'].apply(list).reset_index(name='merged')

In [7]:
def num_answers_for_cat(x):
    return (x.count('Indication: treatment'), x.count('Contraindication'),
                    x.count('Indication: Symptomatic relief'), x.count('Effect'), x.count('I dont know'))

df['votes'] =df.merged.apply(num_answers_for_cat)

In [8]:
aggrements =[]
labels = []
for i, row in df.iterrows():
    aggrements.append( max(row['votes'])/sum(row['votes']))
    idx = row['votes'].index(max(row['votes']))
    if idx==0:
        label = 'treatment'
    elif idx==1:
        label = 'contraindication'
    elif idx==2:
        label = 'relief'
    elif idx==3:
        label = 'effect'
    elif idx==4:
        label = 'IDK'
    labels.append(label)

In [9]:
df['aggrement'] = aggrements
df['relation'] = labels

In [10]:
df_aggred = df[df.aggrement > 0.51].reset_index()
df_aggred = df_aggred[df_aggred.relation != 'IDK'].reset_index()

In [11]:
renamed_columns = { x:x.replace('Input.','') for x  in columns}

In [12]:
renamed_columns

{'Input.Label_ID': 'Label_ID',
 'Input.From': 'From',
 'Input.To': 'To',
 'Input.Type': 'Type',
 'Input.Annotation': 'Annotation',
 'Input.Context': 'Context',
 'Input.DO_ID': 'DO_ID',
 'Input.DB_ID': 'DB_ID',
 'Input.DrugName': 'DrugName',
 'Input.Drug_Brand_Name': 'Drug_Brand_Name',
 'Input.atc_label': 'atc_label',
 'Input.ATC_LEVEL_0': 'ATC_LEVEL_0',
 'Input.DieaseName': 'DieaseName',
 'Input.WordCount': 'WordCount',
 'Input.Set_ID': 'Set_ID',
 'Input.Section': 'Section'}

In [13]:
df_aggred =df_aggred.rename(columns=renamed_columns)

In [14]:
len(df_aggred)

611

In [15]:
df_aggred.sample()

,level_0,index,Label_ID,From,To,Type,Annotation,Context,DO_ID,DB_ID,...,atc_label,ATC_LEVEL_0,DieaseName,WordCount,Set_ID,Section,merged,votes,aggrement,relation
526,526,717,d96823e2-bdbe-4186-95f5-dcfe539640e3.xml,858,868,PREF,THYROIDITIS,Levothyroxine sodium is used for the following...,DOID_7166,DB00451,...,H03AA01,H,thyroiditis,137,dddad621-c42a-45a1-b6af-8f0503f67474,Indication,"[Indication: treatment, Indication: treatment,...","(3, 1, 0, 0, 0)",0.75,treatment


In [16]:
index= df_aggred['Section'] =='Indication'
df_aggred.loc[index,'Section']=  'Indications'

In [17]:
df_aggred = df_aggred[['DieaseName','DO_ID','DB_ID','DrugName','Annotation','Label_ID','Set_ID','Drug_Brand_Name','Context','Section','relation']]

In [18]:
df_aggred.rename(columns={'DieaseName':'disease', 'DrugName':'Drug_name'}, inplace=True)

In [19]:
df_aggred.to_csv('../data/crowd/crowd_2nd.csv')

In [20]:
df_aggred.relation.value_counts()

treatment           356
contraindication    214
relief               29
effect               12
Name: relation, dtype: int64